In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# from bioinfokit.analys import stat
import researchpy as rp
import numpy as np
import datetime
import fiona
from shapely.geometry import shape, Point, Polygon, MultiPolygon, LineString
import shapefile
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.utils import compute_sample_weight

In [12]:
file_path = "./merged_crash_weather.csv"
df_train = pd.read_csv(file_path)

/home/milica/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
df_train = df_train.drop(columns=['ZIP CODE'])

In [14]:
sum_nan = df_train.isnull().sum().to_frame()
sum_nan.columns=['#NaN']
sum_nan_thresh = sum_nan[sum_nan['#NaN'] > 0]
pd.set_option('display.max_rows', 10)
print(sum_nan_thresh)

           #NaN
FeelsLike   176
Wind        749
WindSpeed   749
Pressure    309
Clouds        2


In [15]:
df_train = df_train.dropna()

In [16]:
df_train.head()

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,ON STREET NAME,HOUR,Temperature,FeelsLike,Dew,Humidity,Wind,WindSpeed,Pressure,Precip.,Condition,Day/Night,Clouds,SEVERITY
0,2019-11-24,20:00,MANHATTAN,40.709830,-74.014680,WEST STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0
1,2019-11-24,20:40,QUEENS,40.758960,-73.829850,MAIN STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,1
2,2019-11-24,20:33,BROOKLYN,40.588440,-73.949326,OCEAN AVENUE,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0
3,2019-11-24,20:30,BROOKLYN,40.624798,-73.942650,AVENUE K,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0
4,2019-11-24,20:15,QUEENS,40.663322,-73.854540,79 STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0


In [17]:
# df_train['MINUTE'] = df_train['CRASH TIME'].apply(lambda x: x.split(':')[1])
# df_train['MINUTE'] = df_train['MINUTE'].astype(int)
df_train['YEAR'] = (df_train['CRASH DATE'].apply(lambda x: x.split('-')[0])).astype(int)
df_train['MONTH'] = (df_train['CRASH DATE'].apply(lambda x: x.split('-')[1])).astype(int)
df_train['DAY'] = (df_train['CRASH DATE'].apply(lambda x: x.split('-')[2])).astype(int)

In [18]:
df_train = df_train.drop(columns=['CRASH DATE', 'CRASH TIME'])

In [19]:
df_train.shape

(1024379, 20)

In [20]:
df_train.head()

,BOROUGH,LATITUDE,LONGITUDE,ON STREET NAME,HOUR,Temperature,FeelsLike,Dew,Humidity,Wind,WindSpeed,Pressure,Precip.,Condition,Day/Night,Clouds,SEVERITY,YEAR,MONTH,DAY
0,MANHATTAN,40.709830,-74.014680,WEST STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0,2019,11,24
1,QUEENS,40.758960,-73.829850,MAIN STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,1,2019,11,24
2,BROOKLYN,40.588440,-73.949326,OCEAN AVENUE,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0,2019,11,24
3,BROOKLYN,40.624798,-73.942650,AVENUE K,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0,2019,11,24
4,QUEENS,40.663322,-73.854540,79 STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0,2019,11,24


### Ordinal Encoder

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
categorical_features = ["BOROUGH", "Wind", "Condition", "Clouds", "Day/Night"]
for i, categorical_feature in enumerate(df_train[categorical_features]):
    df_train[categorical_feature] = ord_enc.fit_transform(df_train[[categorical_feature]])

### One-Hot Encoding

In [21]:
pd.set_option('display.max_rows', 30)

one_hot_encoding_data = df_train
categorical_features = ["BOROUGH", "Wind", "Condition", "Clouds", "Day/Night"]
for i, categorical_feature in enumerate(one_hot_encoding_data[categorical_features]):
    temp = pd.get_dummies(one_hot_encoding_data[categorical_feature], prefix = categorical_feature)
    one_hot_encoding_data = one_hot_encoding_data.drop(categorical_feature, axis=1)
    one_hot_encoding_data = pd.concat([one_hot_encoding_data, temp], axis=1) 
df_train = one_hot_encoding_data

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split

y = df_train.SEVERITY
x = df_train.drop(columns=['SEVERITY'])

x, x_test, y, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.11, random_state=42)

In [ ]:
def train_model(x, y):
    class_weights = compute_sample_weight('balanced', y)
    xgb = XGBClassifier(objective="multi:softprob", random_state=42, eval_metric="merror", num_class=3, use_label_encoder=False)    
    xgb_model = xgb.fit(x, y, sample_weight=class_weights)
    return xgb_model

In [ ]:
def test_model(xgb_model, x, y):
    # y = reshape_data(y)
    xgb_predict = xgb_model.predict(x)
    probability = xgb_model.predict_proba(x)
#     roc_auc = roc_auc_score(y, xgb_predict)
    roc_auc = 0
    accuracy = accuracy_score(y, xgb_predict)
    f1 = f1_score(y, xgb_predict, average='micro')
    f1_2 = f1_score(y, xgb_predict, average='weighted')
    print("-----------------------------------------------")
#     print('ROCAUC score:', roc_auc)
    print('Accuracy score:', accuracy)
    print('F1 score:', f1)    
    print('F1 score weighted:', f1)
    return probability, roc_auc, accuracy, f1

In [ ]:
from xgboost import XGBClassifier


xgb_model = train_model(x_train, y_train)
# model_file_name = "./models/Form_"+str(i)+".pkl"
# save_model(model_file_name, xgb_model)


In [ ]:
p, r, a, f = test_model(xgb_model, x_val, y_val)